In [12]:
from scapy.all import *

print(f"Scapy version {conf.version}")
print(f"Interface in use {conf.iface}")
print(f"\nRouting table: \n {conf.route}.")
print(f"\nGateway:", conf.route.route("0.0.0.0")[2])

Scapy version 2.5.0
Interface in use eth0

Routing table: 
 Network          Netmask          Gateway     Iface    Output IP   Metric
0.0.0.0          0.0.0.0          172.16.5.1  eth0     172.16.5.4  100   
127.0.0.0        255.0.0.0        0.0.0.0     lo       127.0.0.1   1     
168.63.129.16    255.255.255.255  172.16.5.1  eth0     172.16.5.4  100   
169.254.169.254  255.255.255.255  172.16.5.1  eth0     172.16.5.4  100   
172.16.5.0       255.255.255.0    0.0.0.0     eth0     172.16.5.4  0     
172.17.0.0       255.255.0.0      0.0.0.0     docker0  172.17.0.1  0     .

Gateway: 172.16.5.1


In [31]:
# see header format of a supported protocols

ls(ICMP)

type       : ByteEnumField                       = ('8')
code       : MultiEnumField (Depends on 8)       = ('0')
chksum     : XShortField                         = ('None')
id         : XShortField (Cond)                  = ('0')
seq        : XShortField (Cond)                  = ('0')
ts_ori     : ICMPTimeStampField (Cond)           = ('55656352')
ts_rx      : ICMPTimeStampField (Cond)           = ('55656352')
ts_tx      : ICMPTimeStampField (Cond)           = ('55656352')
gw         : IPField (Cond)                      = ("'0.0.0.0'")
ptr        : ByteField (Cond)                    = ('0')
reserved   : ByteField (Cond)                    = ('0')
length     : ByteField (Cond)                    = ('0')
addr_mask  : IPField (Cond)                      = ("'0.0.0.0'")
nexthopmtu : ShortField (Cond)                   = ('0')
unused     : MultipleTypeField (ShortField, IntField, StrFixedLenField) = ("b''")
